In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
NFLX_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/NFLX_predictions.csv", index_col=0)


In [3]:
NFLX_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,1
3,2019-04-30,1
4,2019-03-29,0


In [4]:
NFLX_predictions.Date = pd.to_datetime(NFLX_predictions.Date, format="%Y-%m-%d")

In [5]:
NFLX_predictions['Predictions'] = NFLX_predictions['Predictions'].shift(-1)


In [6]:
NFLX_predictions = NFLX_predictions.dropna()


In [7]:
NFLX_predictions = NFLX_predictions[::-1]

In [8]:
NFLX_predictions.head(10)

,Date,Predictions
112,2010-03-31,1.0
111,2010-04-30,0.0
110,2010-05-28,1.0
109,2010-06-30,1.0
108,2010-07-30,1.0
107,2010-08-31,1.0
106,2010-09-30,0.0
105,2010-10-29,0.0
104,2010-11-30,0.0
103,2010-12-31,0.0


In [9]:
NFLX_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_monthly.csv", index_col =0)

In [10]:
NFLX_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
NFLX_predictions = NFLX_predictions.set_index("Date")

In [12]:
NFLX_monthly.tail()

,Open,High,Low,Close,Volume
2002-10-31,9.930,10.30,4.85,8.971,12093806
2002-09-30,13.110,13.67,9.65,9.700,3984231
2002-08-30,13.301,13.99,9.89,13.110,4635214
2002-07-31,14.000,18.19,11.90,13.350,5258635
2002-06-28,15.120,17.25,11.79,13.990,5357226


In [13]:
NFLX_final_df = pd.merge(NFLX_predictions, NFLX_monthly, right_index=True, left_index=True)

In [14]:
NFLX_final_df.index = pd.to_datetime(NFLX_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short!=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(NFLX_final_df, Binary_Classification_1d, cash=10000, commission=.09)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        3409 days 00:00:00
Exposure [%]                               97.3306
Equity Final [$]                            161612
Equity Peak [$]                             211021
Return [%]                                 1516.12
Buy & Hold Return [%]                      338.012
Max. Drawdown [%]                         -69.2941
Avg. Drawdown [%]                         -22.9449
Max. Drawdown Duration           823 days 00:00:00
Avg. Drawdown Duration           225 days 00:00:00
# Trades                                        39
Win Rate [%]                               48.7179
Best Trade [%]                             181.813
Worst Trade [%]                           -46.3889
Avg. Trade [%]                             12.5262
Max. Trade Duration              397 days 00:00:00
Avg. Trade Duration               86 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/CNN/NFLX_CNN_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,8147.295615
2010-06-30,7591.105127
2010-07-30,7164.913306
2010-08-31,8769.770046
2010-09-30,11329.715668
2010-10-29,9462.881400
2010-11-30,7390.572549
2010-12-31,9326.351308
2011-01-31,6866.245719
